## Tezio Wallet

Welcome to Tezio Wallet, an Arduino-based hardware wallet for the Tezos blockchain. Tezio Wallet is compatible with the Arduino MKR WiFi 1010 and the Arduino Nano 33 IoT, both of which include a cryptographic coprocessor to securely store keys and perform certain crytpographic functions. 

###  Installation

Install the Arduino IDE. Download and move the `TezioWallet` folder to your Arduino libraries folder, which is usually `My Documents\Arduino\libraries` on Windows or `Documents\Arduino\libraries` on macOS. Open the Arduino IDE and install the following dependencies using `Tools > Manage Libraries...`.

- ArduinoECCX08
- Crypto
- micro-ecc

### Usage

#### Setup

Running Tezio Wallet on your Arduino device requires that the device first be configured, provisioned, and locked. This is done using the `Tezio_Wallet_Setup.ino` sketch. The sketch runs an interactive setup process using the Arduino IDE's Serial Monitor to share data with the user and get user inputs. The process begins by loading default configuration data onto the Arduino's cryptographic coprocessor. Once the configuration data is written to the device, the user has the option to lock the cofiguration zone. This must be done before the device can be used. Note that the default configuration data stored in the `configuration.h` file is set up to enable current functionality but also to allow for possible future functionality such as encrypted writes to certain slots of the cryptochip's data zone. After the device is configured, the sketch proceeds to derive HD wallet cryptographic keys from a user supplied mnemonic phrase specifice in the `secrets.h` file, or if a mnemonic phrase isn't provided the sketch proceeds to derive a new 24 word phrase using entropy provided by the cryptochip's true random number generator. Mnemonic and key derivation are carried out using specifications outlined in the BIP-0039, BIP-0032, BIP-0044, SLIP-0044, and SLIP-0010. Secret keys are derived for all three elliptic curves supported by the Tezos blockchain: Ed25519, Secp256k1, and NIST P256 (Secp256r1). The keys are written to the Arduino's cryptochip. A user supplied read/write key is also written to the device. The read/write key will allow the user to perform encrypted reads and writes to certain data slots of the device after it is locked. After keys are written, the user is given the option to lock the cryptochip's data zone. After the data zone is locked, clear writes of cryptographic secrets will no longer be supported. The device must be locked before use. 

#### API

After the device is setup, provisioned, and locked, upload the `Tezio_Wallet_API.ino` sketch. The sketch can be run in debug (interactive) mode using the Arduino IDE's serial monitor. However, setting the debug flag to false puts the device into listening mode. It can then be connected via USB to any host machine and recieve and send data via serial. The API sketch invokes the TezioHSM_API class to expose certain cryptographic tools to the host device. Importantly, private (secret) keys never leave the device. In fact, the cryptochip implements hardware support for cryptographic functions using the NIST P256 curve so the NIST P256 secret key never leaves the cryptochip's secure element. This hardware support also means that cryptographic functions involving the NIST P256 curve are much faster than those of the other supported curves. Below are details about the structure of data packets sent and received using the API. This if followed by example interactions with a Tezio Wallet using python. 

### Packet Structure

#### To Wallet

Packets of bytes sent to the hardware wallet have four parts, one prefix byte, two length byte (LSB first), one or more body bytes, and two checksum bytes.

`packet = prefix (1 byte) + length (2 byte) + body (1 or more bytes) + checksum (2 bytes)`

The body is composed of an operation code (opCode), parameters, and data. 

`body = opCode (1 byte) + param1 (1 byte) + param2 (1 byte) + param3 (2 bytes) + data (1 or more bytes)`

A call may not require all parameters but if data is part of the body then values for all parameters must also be included. Parameter 3 is represented in code as a 16-bit variable but is always sent as two bytes with the LSB first. Packets are constructed as follows. First the body is constructed. The length bytes are the length of the body plus 3 to count both the length byte itself and the checksum bytes. The crc16 checksum is then computed for the length and body bytes. The checksum is appended LSB first. The prefix, which serves as a listening byte for the hardware wallet, is always 0x03. 

#### From Wallet

Packets of bytes received from the hardware wallet are similar but do not include a prefix since the host does not need to listen but simply waits for a reply to be sent. The body of the reply depends on the operation being executed. 

### Example Tezio Wallet Interactions using Python

At the time of writing, the Tezio Wallet API implements the following three operations:
- op_get_pk: Query the wallet for a public key corresonding to one of the secret private keys stored on the cryptochip. The public key returned can be raw bytes, compressed, base58 encoded, or as a Tezos public key hash (address). 
- op_sign: Send a message to the wallet for signing. The message can be raw bytes or pre-hashed by the host maching. The signature returned can be raw bytes or base58 encoded.
- op_verify: Send a message and signature to the wallet for signature verification. The message can be raw bytes or pre-hashed and the signature can be raw bytes or base58 encoded. 

Each of these operations is demonstrated below.

In [1]:
# import some useful python tools
import serial
import serial.tools.list_ports
from time import sleep

In [2]:
# some useful functions for interacting with the wallet
def crc16(_data: bytes, reg: int = 0x0000, poly: int = 0x8005) -> int:
    if (_data == None):
        return 0
    
    for octet in _data:
        for i in range(8):
            msb = reg & 0x8000
            if octet & (0x80 >> i):
                msb ^= 0x8000
            reg <<= 1
            if msb:
                reg ^= poly
        reg &= 0xFFFF
    
    return reg

def buildPacket(prefix: bytes, opCode: bytes, param1: bytes = None, param2: bytes = None, param3: int = None, data: bytearray = None) -> bytearray: 
    
    packetLength = 5; # minimum length is one length byte, one opCode byte, and two checksum bytes
    body = [opCode]
    
    if (param1 is not None):
        packetLength+=1
        body+=[param1]
    if (param2 is not None):
        packetLength+=1
        body+=[param2]
    if (param3 is not None):
        packetLength+=2 # int will be represented as two bytes with LSB first
        body+=[param3 & 0xFF, param3 >> 8]
    if (data is not None):
        packetLength+=len(data)
        body+=data
        
    body = [packetLength & 0xFF, packetLength >> 8] + body
    
    checkSum = crc16(body)
    
    packet = bytearray([prefix] + body + [checkSum & 0xFF, checkSum >> 8])
    
    return packet 


def findArduinoPort() -> str:
    ports = serial.tools.list_ports.comports()
    com = None
    for each in ports:
        port = str(each)
        if 'Arduino' in port:
            com = port.split(' ')[0]
    return com

def openSerial(com: str, baud: int = 57600) -> serial.Serial:
    ser = serial.Serial(com, baud)
    if (not ser.is_open):
        return None
    return ser

def sendPacket(ser: serial.Serial, packet: bytearray) -> int:
    if (ser.write(packet) == 0):
        return 0
    return 1

def getReply(ser: serial.Serial) -> bytearray:
    response = bytearray([])
    retries = 500
    for each in range(retries):
        if ser.in_waiting > 0:
            break
        else:
            sleep(0.02)
    if (ser.in_waiting == 0):
        return None
    else:
        while (ser.in_waiting > 0):
            response += ser.read()
        return response

def closeSerial(ser: serial.Serial) -> int:
    ser.close()
    return 1

def validateReply(reply: bytearray) -> int:
    
    checkSum = crc16(reply[:-2]) # last two bytes are checksum
    if (checkSum & 0xFF == reply[-2] and checkSum >> 8 == reply[-1] and reply[0] == len(reply)):
        return 1
    else:
        return 0
    
def parseReply(reply: bytearray) -> bytearray:
    if (validateReply(reply) == 0):
        return 0
    else:
        return reply[2:-2] # chop off the length bytes and the two checksum bytes
    
def queryWallet(packet: bytearray) -> bytearray:  
    com = findArduinoPort()
    if (com is None):
        return 0
    ser = openSerial(com)
    if (ser is None):
        return 0
    if (sendPacket(ser, packet) == 0):
        return 0
    sleep(0.02) # short wait
    response = getReply(ser)
    if (response is None):
        return 0
    else:
        return response

### Get Public Key Operation (op_get_pk)

| Packet Vars | Value |
|-------------|-------| 
| opCode      | 0x11  |
| param1      | curve |
| param2      | mode  |
| param3      | -     |
| data        | -     |


| curve | ECC curve |
|-------|-----------|
| 0x01  | Ed25519   |
| 0x02  | Secp256k1 |
| 0x03  | NIST P256 |

| mode | Public Key Format           |
|------|-----------------------------|
| 0x01 | Raw (32 or 64 bytes)        |
| 0x02 | Compressed (32 or 33 bytes) |
| 0x03 | Base58 Checksum Encoded     |
| 0x04 | Hashed (Tezos Address)      |

In [3]:
# retrieve the public key for curve NIST P256 in compressed format
opCode = 0x11
param1 = 0x03 
param2 = 0x02
packet = buildPacket(0x03, opCode, param1, param2);
print('Packet to be sent...')
print(packet.hex())
print()

reply = queryWallet(packet)
print('Raw reply bytes...')
print(reply.hex())
print()

key = parseReply(reply)
print('Compressed key from the body of the reply...')
print(key.hex())

Packet to be sent...
030700110302300a

Raw reply bytes...
250003105a7d89a3f6c5b3691dd055944556c9858041f86da391b01c8389115b5209f6e1fc

Compressed key from the body of the reply...
03105a7d89a3f6c5b3691dd055944556c9858041f86da391b01c8389115b5209f6


In [4]:
# retrieve the public key hash for the Ed25519 curve 
opCode = 0x11
param1 = 0x01 
param2 = 0x04
packet = buildPacket(0x03, opCode, param1, param2);
print('Packet to be sent...')
print(packet.hex())
print()

reply = queryWallet(packet)
print('Raw reply bytes...')
print(reply.hex())
print()

key = parseReply(reply)
print('Decoded key from the body of the reply...')
print(key.decode('utf-8'))

Packet to be sent...
0307001101042786

Raw reply bytes...
2800747a315265634878775a4a6d5a593464797071324b6d76784b59675542474a6365483176eada

Decoded key from the body of the reply...
tz1RecHxwZJmZY4dypq2KmvxKYgUBGJceH1v


### Sign Operation (op_sign)

*param3 is not used but a value must be included whenever data is sent as part of the packet

| Packet Vars | Value  |
|-------------|--------| 
| opCode      | 0x21   |
| param1      | curve  |
| param2      | mode   |
| param3      | 0x0000 |
| data        | message|


| curve | ECC curve |
|-------|-----------|
| 0x01  | Ed25519   |
| 0x02  | Secp256k1 |
| 0x03  | NIST P256 |

| mode | message hashed | signature format        |
|------|----------------|-------------------------|
| 0x01 | yes            | Raw (64 bytes)          |
| 0x02 | yes            | Base58 Checksum Encoded |
| 0x03 | no             | Raw (64 bytes)          |
| 0x04 | no             | Base58 Checksum Encoded |

### Verify Operation (op_verify)

| Packet Vars | Value              |
|-------------|--------------------| 
| opCode      | 0x22               |
| param1      | curve              |
| param2      | mode               |
| param3      | message length     |
| data        | message + signature|

| curve | ECC curve |
|-------|-----------|
| 0x01  | Ed25519   |
| 0x02  | Secp256k1 |
| 0x03  | NIST P256 |

| mode | message hashed | signature format        |
|------|----------------|-------------------------|
| 0x01 | yes            | Raw (64 bytes)          |
| 0x02 | yes            | Base58 Checksum Encoded |
| 0x03 | no             | Raw (64 bytes)          |
| 0x04 | no             | Base58 Checksum Encoded |

In [5]:
# sign an unhased message using the Secp256k1 curve and getting a base58 checksum endoced result
opCode = 0x21
param1 = 0x02
param2 = 0x04
param3 = 0x0000 # not used but needed in packet since data is included
data = bytearray('This is my message. There are many like it but this is mine', 'utf-8') # 32 bytes so hashed or unhashed mode works
packet = buildPacket(0x03, opCode, param1, param2, param3, data);
print('Packet to be sent...')
print(packet.hex())
print()

reply = queryWallet(packet)
print('Raw reply bytes...')
print(reply.hex())
print()

sig = parseReply(reply)
print('Base58 encoded signature from the body of the reply...')
print(sig.decode('utf-8'))

Packet to be sent...
034400210204000054686973206973206d79206d6573736167652e20546865726520617265206d616e79206c696b65206974206275742074686973206973206d696e65dcba

Raw reply bytes...
67007370736967315065616734706e484d615a786936506b506548474a34397842744d394645484e7843534b706e506f71454d42703847705537557248336163437856326932366a6779506f765a504a6b6631623532754c766a674441706a367472614257456d

Base58 encoded signature from the body of the reply...
spsig1Peag4pnHMaZxi6PkPeHGJ49xBtM9FEHNxCSKpnPoqEMBp8GpU7UrH3acCxV2i26jgyPovZPJkf1b52uLvjgDApj6traBW


In [6]:
# verify the signature
opCode = 0x22
# param1 and param2 are unchanged
param3 = len(data) 
data = list(data) + list(sig) # the data is not the message signed with the signature appended
packet = buildPacket(0x03, opCode, param1, param2, param3, data);
print('Packet to be sent...')
print(packet.hex())
print()

reply = queryWallet(packet)
print('Raw reply bytes...')
print(reply.hex())
print()

valid = parseReply(reply)
print('Signature valid (0x01) or invalid (0x00)...')
print(valid.hex())

Packet to be sent...
03a7002202043b0054686973206973206d79206d6573736167652e20546865726520617265206d616e79206c696b65206974206275742074686973206973206d696e657370736967315065616734706e484d615a786936506b506548474a34397842744d394645484e7843534b706e506f71454d42703847705537557248336163437856326932366a6779506f765a504a6b6631623532754c766a674441706a36747261425733ea

Raw reply bytes...
0500014180

Signature valid (0x01) or invalid (0x00)...
01
